In [1]:
import fitz
def pdf_extract(pdf_path):
    pdf_document = fitz.open(pdf_path)
    text = ""
    for page_num in range(pdf_document.page_count):
        page = pdf_document.load_page(page_num)
        text += page.get_text()
    return text
ssu = pdf_extract('Ssudan/ssudan_7.pdf')+" "+pdf_extract('Ssudan/ssudan_8.pdf')+" "+pdf_extract('Ssudan/ssudan_9.pdf')
eth = pdf_extract('ethiopia/eth_7.pdf') + " " + pdf_extract('ethiopia/eth_8.pdf') + " " + pdf_extract('ethiopia/eth_9.pdf')
sa = pdf_extract('southAfri/sa_7.pdf')+" "+pdf_extract('southAfri/sa_9.pdf')

In [2]:
textbooks = [ssu,eth,sa]

# Data Cleaning

In [3]:
import re
from langdetect import detect
from langdetect import LangDetectException
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [4]:
def clean_text(post):
    if detect(post) != 'en':
        return ''
    # Make posts lowercase
    post = post.lower()

    # Remove punctuation
    post = re.sub(r'[^a-zA-Z\s]', ' ', post)
    
    # Remove words with repeated letters
    #post = re.sub(r'([a-zA-Z])\1+', r'\1', post)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(post)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    
    return ' '.join(filtered_text)

In [5]:
cleanTextbooks = []
for i in textbooks:
    cleanTextbooks.append(clean_text(i))

In [6]:
#These have spaces, we use these to select the male and female words
menWords = [" man ", " boy ", " male ", " brother ", " father ", " son ", " husband ", " king ", " prince ", " uncle ", " nephew ", " he ", " him ", " his ", " gentleman ", " sir ", " mr. ", " hero ", " lord ", " patriarch ", " men "]
womenWords = [" woman ", " girl ", " female ", " sister ", " mother ", " daughter ", " wife ", " queen ", " princess ", " aunt ", " niece ", " she ", " her ", " hers ", " lady ", " ma'am "," madam ", " mrs. ", " ms. ", " miss ", " heroine ", " dame ", " matriarch ", " women "]
#These have no spaces, so these can be used to throw away words during TFIDF
menWordsNoSpace = ["man", "boy", "male", "brother", "father", "son", "husband", "king", "prince", "uncle", "nephew", "he", "him", "his", "gentleman", "sir", "mr", "hero", "lord", "patriarch", "men"]
womenWordsNoSpace = ["woman", "girl", "female", "sister", "mother", "daughter", "wife", "queen", "princess", "aunt", "niece", "she", "her", "hers", "lady", "ma'am","madam", "mrs", "heroine", "dame", "matriarch", "women"]

In [7]:
menPara = [[] for _ in range(3)]
womenPara = [[] for _ in range(3)]
for i in range(0, 3):
    text = cleanTextbooks[i]
    paragraphs = [text[j:j+100] for j in range(0, len(text), 100)]
    for paragraph in paragraphs:
        for j in menWords:
            if j in paragraph:
                menPara[i].append(paragraph)
        for j in womenWords:
            if j in paragraph:
                womenPara[i].append(paragraph)

# Occurence

In [8]:
men_count = sum(len(sublist) for sublist in menPara)
print(men_count)
women_count = sum(len(sublist) for sublist in womenPara)
print(women_count)

728
608


# Firstness

In [9]:
femaleFirst = []
for sublist in womenPara:  
    for paragraph in sublist:  
        if isinstance(paragraph, str):  
            words = paragraph.split()  
            for i in range(len(words) - 1): 
                if words[i].lower() in womenWordsNoSpace and words[i + 1].lower() in menWordsNoSpace:
                    femaleFirst.append((words[i], words[i + 1]))
print(len(femaleFirst),femaleFirst)

54 [('women', 'men'), ('mother', 'father'), ('mother', 'father'), ('women', 'men'), ('sister', 'uncle'), ('women', 'men'), ('girl', 'father'), ('girl', 'father'), ('mother', 'father'), ('mother', 'father'), ('wife', 'husband'), ('woman', 'husband'), ('women', 'men'), ('wife', 'son'), ('daughter', 'brother'), ('wife', 'son'), ('daughter', 'brother'), ('wife', 'son'), ('daughter', 'brother'), ('wife', 'son'), ('daughter', 'brother'), ('woman', 'man'), ('girl', 'boy'), ('wife', 'husband'), ('sister', 'brother'), ('daughter', 'son'), ('woman', 'man'), ('girl', 'boy'), ('wife', 'husband'), ('sister', 'brother'), ('daughter', 'son'), ('woman', 'man'), ('girl', 'boy'), ('wife', 'husband'), ('sister', 'brother'), ('daughter', 'son'), ('woman', 'man'), ('girl', 'boy'), ('wife', 'husband'), ('sister', 'brother'), ('daughter', 'son'), ('woman', 'man'), ('girl', 'boy'), ('wife', 'husband'), ('sister', 'brother'), ('daughter', 'son'), ('woman', 'man'), ('girl', 'boy'), ('wife', 'husband'), ('sister

In [10]:
maleFirst = []
for sublist in menPara:  
    for paragraph in sublist:  
        if isinstance(paragraph, str):  
            words = paragraph.split()  
            for i in range(len(words) - 1):  
                if words[i].lower() in menWordsNoSpace and words[i + 1].lower() in womenWordsNoSpace:
                    maleFirst.append((words[i], words[i + 1]))
print(len(maleFirst),maleFirst)

75 [('boy', 'mother'), ('sir', 'madam'), ('sir', 'madam'), ('sir', 'madam'), ('father', 'girl'), ('man', 'woman'), ('men', 'women'), ('brother', 'sister'), ('brother', 'sister'), ('father', 'daughter'), ('father', 'daughter'), ('brother', 'sister'), ('husband', 'wife'), ('men', 'women'), ('man', 'woman'), ('man', 'woman'), ('man', 'woman'), ('men', 'women'), ('men', 'women'), ('men', 'women'), ('sir', 'madam'), ('men', 'women'), ('men', 'women'), ('men', 'women'), ('boy', 'girl'), ('brother', 'sister'), ('brother', 'sister'), ('men', 'women'), ('husband', 'wife'), ('brother', 'sister'), ('man', 'wife'), ('man', 'wife'), ('husband', 'daughter'), ('man', 'woman'), ('brother', 'sister'), ('brother', 'wife'), ('brother', 'sister'), ('brother', 'wife'), ('brother', 'sister'), ('brother', 'wife'), ('man', 'woman'), ('son', 'daughter'), ('son', 'daughter'), ('man', 'girl'), ('man', 'girl'), ('boy', 'wife'), ('father', 'mother'), ('brother', 'daughter'), ('man', 'girl'), ('boy', 'wife'), ('fat

In [11]:
sameLevel = {
    "siblings": {"brother", "sister"},
    "parent_child": {"father", "mother", "son", "daughter"},
    "spouses": {"husband", "wife"},
    "royalty": {"king", "queen", "prince", "princess"},
    "extended_family": {"uncle", "aunt", "nephew", "niece"},
    "personal_pronouns": {"he", "she", "him", "her", "his", "hers"},
    "titles": {"gentleman", "lady", "sir", "ma'am", "mr", "mrs", "madam"},
    "heroic_roles": {"hero", "heroine"},
    "social_roles": {"patriarch", "matriarch"},
    "general_plural": {"men", "women"},
}
flat = {word for group in sameLevel.values() for word in group}
def isSame(pair):
    return pair[0] in flat and pair[1] in flat
femaleFirst = [pair for pair in femaleFirst if isSame(pair)]

In [12]:
maleFirst = [pair for pair in maleFirst if isSame(pair)]
print(len(maleFirst))
print(len(femaleFirst))

46
38


# Sentiment analysis

In [13]:
fm = ', '.join(map(str, [item for sublist in menPara for item in sublist]))
fw = ', '.join(map(str, [item for sublist in womenPara for item in sublist]))
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
fscores = analyzer.polarity_scores(fw)
mscores = analyzer.polarity_scores(fm)
print(fscores,mscores)

{'neg': 0.066, 'neu': 0.776, 'pos': 0.158, 'compound': 1.0} {'neg': 0.091, 'neu': 0.758, 'pos': 0.152, 'compound': 1.0}


# TFIDF

In [14]:
from langdetect import detect
from langdetect import LangDetectException
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
#stemming the words
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
def stem_nested_list(nested_list):
    return [
        [
            ' '.join(stemmer.stem(word) for word in sentence.split())
            for sentence in sublist
        ]
        for sublist in nested_list
    ]
menParaOriginal = menPara
womenParaOriginal = womenPara
menPara = stem_nested_list(menPara)
womenPara = stem_nested_list(womenPara)
fm = ', '.join(map(str, [item for sublist in menPara for item in sublist]))
fw = ', '.join(map(str, [item for sublist in womenPara for item in sublist]))

In [16]:
tobeTFIDF = [" ".join(" ".join(row) for row in menPara)," ".join(" ".join(row) for row in womenPara)]
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(tobeTFIDF)
feature_names = vectorizer.get_feature_names_out()
dense_tfidf = tfidf_matrix.toarray()

In [17]:
#tfidf of female words
doc_index = 1
doc_tfidf_scores = dense_tfidf[doc_index]
word_score_pairs = list(zip(feature_names, doc_tfidf_scores))
sorted_word_scores = sorted(word_score_pairs, key=lambda x: x[1], reverse=True)
top_300_words_fs = sorted_word_scores[:300]
top_300_words_f = [word for word, score in sorted_word_scores[:300]]

In [18]:
#tfidf of male words
doc_index = 0  
doc_tfidf_scores = dense_tfidf[doc_index]
word_score_pairs_m = list(zip(feature_names, doc_tfidf_scores))
sorted_word_scores_m = sorted(word_score_pairs_m, key=lambda x: x[1], reverse=True)
top_300_words_ms = sorted_word_scores_m[:300]
top_300_words_m = [word for word, score in sorted_word_scores_m[:300]]

In [19]:
#remove common words and gendered keywords
common = list(set(top_300_words_f) & set(top_300_words_m))
top_300_words_f = [word for word in top_300_words_f if word not in common]
top_300_words_f = [word for word in top_300_words_f if word not in womenWordsNoSpace]
female_300 = " ".join(top_300_words_f)
top_300_words_m = [word for word in top_300_words_m if word not in common]
top_300_words_m = [word for word in top_300_words_m if word not in menWordsNoSpace]
male_300 = " ".join(top_300_words_m)

In [20]:
print(female_300)
print(male_300)

dawit child peter ladi coffe discuss class done femal hope milk today enough sudan atieno chang hear hospit jack lesson mobil partner prepar babi beauti birthday cotton find harri shop st market xavier differ eat flat groom howev lot mr opinion pregnant present sugar travel wash karl smart spider teach compar dinner eye glass import law mean much phone photo practic tomorrow briefcas ch coupl meseret bad born care cat chair cindi cultur daddi danc dog dress dri ed happi hello keep kiss la learner member mine offic plant respect sat seen shout sing uniform wa accid
uncl fi luther okonkwo martin usual abraham run unoka god ill yalla yam dead er pm sampson barn given harvest maureen tear tree villag week mandela nwakibi wizard anoth countri equal five jr put back cloth face join lost met nation possess saw true bite fight pen ali anim bull carri continu farm flower heard holiday hyena indb ing laugh leav sb someth war abl aster cave civil consult hundr kanja neither sea sens short suddenl

In [21]:
from collections import Counter
import re
mwords = male_300.split()
filteredMwords = [word for word in mwords if word in fm]
wordMcounts = Counter(filteredMwords)
topMwords = wordMcounts.most_common(40)

In [22]:
fwords = female_300.split()
filteredFwords = [word for word in fwords if word in fw]
wordFcounts = Counter(filteredFwords)
topFwords = wordFcounts.most_common(40)

In [23]:
from collections import Counter
import re

# Example word lists (assuming male_300 and female_300 are defined as strings)
mwords = male_300.split()  # Male words
fwords = female_300.split()  # Female words

# Filtered words for male
filteredMwords = [word for word in mwords if word in fm]
wordMcounts = Counter(filteredMwords)
topMwords = wordMcounts.most_common(40)
print("Top 40 Male Words:", topMwords)

# Filtered words for female
filteredFwords = [word for word in fwords if word in fw]
wordFcounts = Counter(filteredFwords)
topFwords = wordFcounts.most_common(40)
print("Top 40 Female Words:", topFwords)


Top 40 Male Words: [('uncl', 1), ('fi', 1), ('luther', 1), ('okonkwo', 1), ('martin', 1), ('usual', 1), ('abraham', 1), ('run', 1), ('unoka', 1), ('god', 1), ('ill', 1), ('yalla', 1), ('yam', 1), ('dead', 1), ('er', 1), ('pm', 1), ('sampson', 1), ('barn', 1), ('given', 1), ('harvest', 1), ('maureen', 1), ('tear', 1), ('tree', 1), ('villag', 1), ('week', 1), ('mandela', 1), ('nwakibi', 1), ('wizard', 1), ('anoth', 1), ('countri', 1), ('equal', 1), ('five', 1), ('jr', 1), ('put', 1), ('back', 1), ('cloth', 1), ('face', 1), ('join', 1), ('lost', 1), ('met', 1)]
Top 40 Female Words: [('dawit', 1), ('child', 1), ('peter', 1), ('ladi', 1), ('coffe', 1), ('discuss', 1), ('class', 1), ('done', 1), ('femal', 1), ('hope', 1), ('milk', 1), ('today', 1), ('enough', 1), ('sudan', 1), ('atieno', 1), ('chang', 1), ('hear', 1), ('hospit', 1), ('jack', 1), ('lesson', 1), ('mobil', 1), ('partner', 1), ('prepar', 1), ('babi', 1), ('beauti', 1), ('birthday', 1), ('cotton', 1), ('find', 1), ('harri', 1), (

# random forest

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd
male_segments = fm.split(" ")  
female_segments = fw.split(" ")  
texts = male_segments + female_segments  
labels = [0] * len(male_segments) + [1] * len(female_segments)  
vectorizer = TfidfVectorizer()  
X = vectorizer.fit_transform(texts)  
feature_names = vectorizer.get_feature_names_out()

clf = RandomForestClassifier(random_state=42, n_estimators=100)
clf.fit(X, labels)


proba = clf.predict_proba(X)[:, 1]  # Probability of being female
female_contributions = np.dot(X.T.toarray(), proba)
female_importance = female_contributions / np.sum(X.toarray(), axis=0)
female_importance_df = pd.DataFrame({
    "Word": feature_names,
    "FemaleImportance": female_importance
}).sort_values(by="FemaleImportance", ascending=False)

print("Top words most related to Female:")
print(female_importance_df.head(10))

Top words most related to Female:
        Word  FemaleImportance
276   atieno               1.0
3514  spider               1.0
924    dawit               1.0
3783    ther               1.0
4258  xavier               1.0
2097    ladi               1.0
3438   smart               1.0
1743    hink               1.0
3727   teach               1.0
2307  market               1.0


In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd
male_segments = fm.split(" ")  
female_segments = fw.split(" ")  
texts = male_segments + female_segments  
labels = [0] * len(male_segments) + [1] * len(female_segments)  
vectorizer = TfidfVectorizer()  
X = vectorizer.fit_transform(texts)  
feature_names = vectorizer.get_feature_names_out()

clf = RandomForestClassifier(random_state=42, n_estimators=100)
clf.fit(X, labels)

proba = clf.predict_proba(X)[:, 0]  # Probability of being male (class 0)


male_contributions = np.dot(X.T.toarray(), proba)
male_importance = male_contributions / np.sum(X.toarray(), axis=0)
male_importance_df = pd.DataFrame({
    "Word": feature_names,
    "MaleImportance": male_importance
}).sort_values(by="MaleImportance", ascending=False)

print("Top words most related to Male:")
print(male_importance_df.head(10))

Top words most related to Male:
          Word  MaleImportance
2038     kanja             1.0
500   brethren             1.0
2288   mandela             1.0
2234      lord             1.0
26      access             1.0
152   although             1.0
4210    wizard             1.0
620       cave             1.0
2261    luther             1.0
4011     unoka             1.0


In [30]:
fm = ', '.join(map(str, [item for sublist in menParaOriginal for item in sublist]))
fw = ', '.join(map(str, [item for sublist in womenParaOriginal for item in sublist]))
ps = PorterStemmer()
words = word_tokenize(fw)
matching = [word for word in words if ps.stem(word) in {"cass", "aborigin","sungen"}]
matching

[]